In [1]:
import cv2
import mediapipe as mp
import numpy as np
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
[x for x in dir(cv2) if x.startswith('COLOR_BGR2RGB')]

['COLOR_BGR2RGB', 'COLOR_BGR2RGBA']

In [2]:
def calculate_angle(a,b,c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)

    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)

    if angle > 180.0:
        angle = 360 - angle

    return angle

In [3]:
def checkForFall(a,b,c):
    #here we check if the person has fallen by comparing the y coordinate distance between the shoulder and ankle and the shoulder and hip.

    a = np.array(a) #shoulder
    b = np.array(b) #hip
    c = np.array(c) 

    status = False

    #r = np.abs((a[1]-b[1])/c[1]-b[1])

    angle = calculate_angle(a,b,c)
    
    return angle
    
    #if(r<0.58 and r>0.50):
     #   print("Fallen Down")
      #  return not status
    
    #else:
     #   return status

In [4]:
def getAccelaration(a,b,c):
    acc = (b-a)*c
    return acc

def getVelocity(a,b,c):
    v = (b-a)*c
    return v

# Fall detection

# TUG Test

In [5]:
import time

cap = cv2.VideoCapture('./assets/tugtest.mp4')
pri

window_name = 'Fall Demo'
# cv2.namedWindow(window_name, cv2.WND_PROP_FULLSCREEN)
# cv2.setWindowProperty(window_name, cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)

fps = cap.get(cv2.CAP_PROP_FPS)
print(fps)

initialCoordinate = 0
initialVelocity = 0
sat = False 
start = True
walked = False
finish = False

t_i = None
t_f = None

with mp_pose.Pose(min_detection_confidence = 0.5, min_tracking_confidence = 0.5) as pose:
    while cap.isOpened():
        cap.set(cv2.CAP_PROP_FRAME_WIDTH, 768)
        cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 1366)
        
        ret, frame = cap.read()
#         cv2.resize(frame, (1366, 768))
        
        if(finish):
#             print("Finished")
            t_f = time.time()
            time.sleep(2)
            break
                
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        results = pose.process(image)

        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS, 
                                  mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2),
                                  mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                  )

        
        try:
            
            landmarks = results.pose_landmarks.landmark

            shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            nose = [landmarks[mp_pose.PoseLandmark.NOSE.value].x,landmarks[mp_pose.PoseLandmark.NOSE.value].y]
            knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
            left_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
            right_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]
            
            
            finalCoordinate = nose[0]
            velocity = getVelocity(initialCoordinate, finalCoordinate, fps)
            acc = getAccelaration(initialVelocity, velocity, fps)
            
            
            initialCoordinate = finalCoordinate
            initialVelocity = velocity
            
            print("Velocity ={}".format(velocity), "Acceleration={}".format(acc))
            
#             cv2.putText(image, "Velocity = {}, Acceleration={}".format(velocity, acc), (10,10), cv2.FONT_HERSHEY,SIMPLEX, 1, (255,255,255), 1, 2)

            if(acc>15 or acc<-15):
                cv2.putText(image, "FALLING DOWN", 
                                    tuple(np.multiply(nose, [640, 480]).astype(int)),
                                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255), 2, cv2.LINE_AA)
#                 print("Falling Down!")
            
            fallenDownAngle = checkForFall(shoulder, hip, [680,0])
            
            if(fallenDownAngle>165 or fallenDownAngle<15):
                cv2.putText(image, "FALLEN DOWN", 
                                    tuple(np.multiply(hip, [640, 480]).astype(int)),
                                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255), 2, cv2.LINE_AA)
            
            
            
            
            #angles between various body segments
            angle_between_nose_hip_knee = calculate_angle(nose, hip, knee)
            angle_between_hip_knee_ankle = calculate_angle(hip, knee, left_ankle)
            
            #stride length
            stride_length = left_ankle[0] - right_ankle[0]
            
            #for sitting posture
            
            if(angle_between_nose_hip_knee < 100 and angle_between_nose_hip_knee > 85
               and angle_between_hip_knee_ankle < 100 and angle_between_hip_knee_ankle > 85):
                sat = True
                if(start):
                    print("Taking initial time")
                    t_i = time.time()
                start = False
                if(walked==False and finish==False):
                    print("Sitting intiially")
                    t_i = time.time()
                    cv2.putText(image, "SITTING INITIALLY", 
                                    tuple(np.multiply(hip, [640, 480]).astype(int)),
                                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255), 2, cv2.LINE_AA)
                if(walked==True):
                     finish = True
                     print("Sitting finally")
                     cv2.putText(image, "SITTING FINALLY", 
                                    tuple(np.multiply(hip, [640, 480]).astype(int)),
                                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255), 2, cv2.LINE_AA)
                
                        
            elif(angle_between_nose_hip_knee > 150 and angle_between_hip_knee_ankle > 150):
                walked = True
                cv2.putText(image, "STANDING", 
                                    tuple(np.multiply(nose, [640, 480]).astype(int)),
                                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255), 2, cv2.LINE_AA)
                if(stride_length > -0.01 and stride_length < 0.01):
                    cv2.putText(image, "WALKING", 
                                    tuple(np.multiply(left_ankle, [640, 480]).astype(int)),
                                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255), 2, cv2.LINE_AA)
            
        except:
            pass

            
        cv2.imshow(window_name, image)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            print("hi")
            t_f = time.time()
            break

    cap.release()
    cv2.destroyAllWindows()
    print(str(t_f-t_i) + "s")

30.005342723472282


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


Velocity =22.562652362140263 Acceleration=677.00011687658


QObject::moveToThread: Current thread (0x55d3c4e25de0) is not the object's thread (0x55d3c52fc9c0).
Cannot move to target thread (0x55d3c4e25de0)

QObject::moveToThread: Current thread (0x55d3c4e25de0) is not the object's thread (0x55d3c52fc9c0).
Cannot move to target thread (0x55d3c4e25de0)

QObject::moveToThread: Current thread (0x55d3c4e25de0) is not the object's thread (0x55d3c52fc9c0).
Cannot move to target thread (0x55d3c4e25de0)

QObject::moveToThread: Current thread (0x55d3c4e25de0) is not the object's thread (0x55d3c52fc9c0).
Cannot move to target thread (0x55d3c4e25de0)

QObject::moveToThread: Current thread (0x55d3c4e25de0) is not the object's thread (0x55d3c52fc9c0).
Cannot move to target thread (0x55d3c4e25de0)

QObject::moveToThread: Current thread (0x55d3c4e25de0) is not the object's thread (0x55d3c52fc9c0).
Cannot move to target thread (0x55d3c4e25de0)

QObject::moveToThread: Current thread (0x55d3c4e25de0) is not the object's thread (0x55d3c52fc9c0).
Cannot move to tar

Velocity =-0.03034655185531841 Acceleration=-677.9106755654744
Taking initial time
Sitting intiially
Velocity =-0.0007314792379081346 Acceleration=0.8886104036659146
Velocity =-0.0009782864135348401 Acceleration=-0.007405533891291513
Velocity =-0.0006885562508426207 Acceleration=0.008693452828907428
Velocity =0.006638755332799501 Acceleration=0.21985849530885032
Velocity =0.020214938450062704 Acceleration=0.4073580273101007
Velocity =0.00035769155887928344 Acceleration=-0.5958234985145631
Sitting intiially
Velocity =0.002473437129650245 Acceleration=0.06348367096665118
Sitting intiially
Velocity =0.0031154934778385586 Acceleration=0.019265120775171396
Sitting intiially
Velocity =0.004220760394775544 Acceleration=0.03316391264360983
Sitting intiially
Velocity =0.0005884026143564212 Acceleration=-0.10899014009574687
Sitting intiially
Velocity =0.0013717471283020521 Acceleration=0.023504520611490467
Sitting intiially
Velocity =0.00475729773309447 Acceleration=0.10158460620445536
Sitting i

In [6]:
# calc accelaration 
# explore more repos where people used mobile camera (github repos)
# https://github.com/Foxpace/BeSafeBox_Android_app
# https://github.com/taufeeque9/HumanFallDetection
# https://github.com/ambianic/fall-detection
# https://github.com/skulkarni307/CCTV_Fall_Detector
# https://github.com/TeCSAR-UNCC/S-RAD-ActionLocalizationClassification
# https://github.com/uttej2001/Image-based-Human-Fall-Detection
# https://github.com/JJN123/Fall-Detection

# https://github.com/cwlroda/falldetection_openpifpaf
# try implmenting the above pifpaf code

# https://youtu.be/7_mcWCB76Ps
# 

## https://github.com/search?l=Python&q=fall+detection&type=Repositories



gait score speed, stance, balance to be calculated  (parameters)
use the above parameters to predict the chances of fall

tug score (tug test) : https://www.google.com/search?channel=fs&client=ubuntu&q=TUG+test
use the above data to check chances of fall, etc

health parameters: post stroke, alzeihmers, parkinsons(other neurogenerative diseases)
check for things related to rehabilitations 
aim to reduce manual load for hospitals 

(search for app developers: java/kotlin for mobile apps)